# 5. Write a program to implement the naïve Bayesian classifier for a sample training data set stored as a .CSV file. Compute the accuracy of the classifier, considering few test data sets.

## Read and Handle Data

In [1]:
import csv
import random
import math

random.seed(0)
# 1.Data Handling
# 1.1 Loading the Data from csv file of Pima indians diabetes dataset.
def loadcsv(filename):
    reader = csv.reader(open(filename,"r"))
    dataset =[]
    for row in reader:
        inlist=[]
        for i in range(len(row)):
            # converting the attributes from string to floating point numbers
            inlist.append(float(row[i]))
        dataset.append(inlist)
    return dataset

#### Splitting the Data set into Training Set

In [2]:
#The naive bayes model is comprised of a 
#summary of the data in the training dataset. 
#This summary is then used when making predictions.
#involves the mean and the standard deviation for each attribute, by class value
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy)) # random index
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

## Summarize Data

#### Separate Data By Class

In [3]:
#Function to categorize the dataset in terms of classes 
#The function assumes that the last attribute (-1) is the class value. 
#The function returns a map of class values to lists of data instances.
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated
 

#### Calculate the Mean

In [4]:
#The mean is the central middle or central tendency of the data, 
# and we will use it as the middle of our gaussian distribution 
# when calculating probabilities

#2.2 : Calculate Mean
def mean(numbers):
    return sum(numbers)/float(len(numbers))

#### Calculate the Standard Deviation

In [7]:
#The standard deviation describes the variation of spread of the data, 
#and we will use it to characterize the expected spread of each attribute
#in our Gaussian distribution when calculating probabilities.

#2.3 : Calculate Standard Deviation
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

#### Summarize Dataset

In [8]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

#### Summarize Attributes By Class

In [9]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    #print(list(separated.items()))
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

## Make Prediction

#### Calculate Probaility Density Function

In [10]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent


#### Calculate Class Probabilities

In [12]:
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *=calculateProbability(x, mean, stdev)
    return probabilities

#### Prediction : look for the largest probability and return the associated class

In [13]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, 
                                                inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

#### Make Predictions

In [14]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

#### Computing Accuracy

In [15]:
def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

#### Main Function

In [16]:
def main():
    filename = 'PI_Diabetes.csv'
    splitRatio = 0.67
    dataset = loadcsv(filename)
    
    #print("\n The Data Set :\n",dataset)
    print("\n The length of the Data Set : ",len(dataset))
    
    print("\n The Data Set Splitting into Training and Testing \n")
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    
    print('\n Number of Rows in Training Set:{0} rows'
          .format(len(trainingSet)))
    print('\n Number of Rows in Testing Set:{0} rows'
          .format(len(testSet)))
    
    print("\n First Five Rows of Training Set:\n")
    for i in range(0,5):
        print(trainingSet[i],"\n")
    
    print("\n First Five Rows of Testing Set:\n")
    for i in range(0,5):
        print(testSet[i],"\n")
   
    # prepare model
    summaries = summarizeByClass(trainingSet)
    print("\n Model Summaries:\n",summaries)
   
    # test model
    predictions = getPredictions(summaries, testSet)
    print("\nPredictions:\n",predictions)
    
    accuracy = getAccuracy(testSet, predictions)
    print('\n Accuracy: {0}%'.format(accuracy))
main()



 The length of the Data Set :  768

 The Data Set Splitting into Training and Testing 


 Number of Rows in Training Set:514 rows

 Number of Rows in Testing Set:254 rows

 First Five Rows of Training Set:

[4.0, 158.0, 78.0, 0.0, 0.0, 32.9, 0.803, 31.0, 1.0] 

[3.0, 89.0, 74.0, 16.0, 85.0, 30.4, 0.551, 38.0, 0.0] 

[7.0, 133.0, 84.0, 0.0, 0.0, 40.2, 0.696, 37.0, 0.0] 

[0.0, 138.0, 0.0, 0.0, 0.0, 36.3, 0.933, 25.0, 1.0] 

[3.0, 116.0, 74.0, 15.0, 105.0, 26.3, 0.107, 24.0, 0.0] 


 First Five Rows of Testing Set:

[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0, 1.0] 

[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0, 0.0] 

[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0, 1.0] 

[0.0, 118.0, 84.0, 47.0, 230.0, 45.8, 0.551, 31.0, 1.0] 

[7.0, 107.0, 74.0, 0.0, 0.0, 29.6, 0.254, 31.0, 1.0] 


 Model Summaries:
 {1.0: [(4.701754385964913, 3.749344627974186), (142.9298245614035, 31.184947150709903), (68.81871345029239, 23.193226713717014), (22.239766081871345, 17.934713233516998), (110.0